In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Charger le fichier de Kaggle qui contient les ventes réelles
df_sales = pd.read_excel('Pizza_Sales.xlsx')

# Calculer la distribution des pizzas
pizza_distribution = df_sales['pizza_name'].value_counts(normalize=True)


In [ ]:
# Paramètres de simulation
num_clients = 70000  
mean_pizzas_per_client = 7
std_pizzas_per_client = 3  

#  Utiliser une distrib normale pour générer le nombre de pizzas par clients à l'année
np.random.seed(42)  
pizzas_per_client = np.clip(
    np.random.normal(loc=mean_pizzas_per_client, scale=std_pizzas_per_client, size=num_clients),
    2, 20
).astype(int)

In [ ]:
# Générer les commandes pour chaque client
data = []
for client_id, num_pizzas in enumerate(pizzas_per_client, start=1):
    
    favorite_pizzas = pizza_distribution.sample(n=min(2, len(pizza_distribution)), weights=pizza_distribution.values).index.tolist()
    
    for _ in range(num_pizzas):
        order_date = datetime.now() - timedelta(days=random.randint(1, 365))
        
        if num_pizzas <= 6:  #Clients pas fréquents
            if random.random() < 0.8:  # 80% chance de pizza favorite
                pizza_name = random.choice(favorite_pizzas)
            else:  # Choisir une pizza aux ingrédients similaires à celles d'avant
                similar_pizzas = df_sales[
                    df_sales['pizza_ingredients'].str.contains(
                        '|'.join(df_sales[df_sales['pizza_name'].isin(favorite_pizzas)]['pizza_ingredients'].iloc[0].split(', '))
                    )
                ]
                if not similar_pizzas.empty:
                    pizza_name = similar_pizzas.sample(1)['pizza_name'].iloc[0]
                else:  
                    pizza_name = df_sales.sample(1)['pizza_name'].iloc[0]
        else:  # Clients fréquents
            if random.random() < 0.25:  # 25% chance de pizza favorite
                pizza_name = random.choice(favorite_pizzas)
            else:  # Choisir parmi la distribution générale
                pizza_name = pizza_distribution.sample(n=1, weights=pizza_distribution.values).index[0]
        
        # Récup les ingrédients
        ingredients = df_sales[df_sales['pizza_name'] == pizza_name]['pizza_ingredients'].iloc[0]
        
        # Ajouter l'entrée
        data.append({
            'client_id': client_id,
            'pizza_name': pizza_name,
            'order_date': order_date,
            'ingredients': ingredients
        })

# Convertir les données en DataFrame
df_consumers = pd.DataFrame(data)

# Sauvegarder le fichier généré
df_consumers.to_csv('realistic_synthetic_consumer_data.csv', index=False, encoding='utf-8')


In [6]:
# Calculer le nombre total de pizzas par client
pizza_count_per_client = df_consumers.groupby('client_id').size()

# Compter combien de clients ont consommé chaque quantité
distribution = pizza_count_per_client.value_counts().sort_index()

# Afficher les résultats
distribution_df = distribution.reset_index()
distribution_df.columns = ['Number of Pizzas', 'Number of Clients']

print(distribution_df)

    Number of Pizzas  Number of Clients
0                  2                658
1                  3                458
2                  4                659
3                  5                824
4                  6                920
5                  7                917
6                  8                819
7                  9                643
8                 10                470
9                 11                291
10                12                177
11                13                 92
12                14                 44
13                15                 16
14                16                  9
15                17                  1
16                18                  2


In [7]:
# Calculer combien de clients uniques ont consommé chaque pizza
pizza_client_count = df_consumers.groupby('pizza_name')['client_id'].nunique()

# Convertir en DataFrame pour une meilleure lisibilité
pizza_client_distribution = pizza_client_count.reset_index()
pizza_client_distribution.columns = ['Pizza Name', 'Number of Clients']

# Trier par le nombre de clients
pizza_client_distribution = pizza_client_distribution.sort_values(by='Number of Clients', ascending=False)

print(pizza_client_distribution)

                                    Pizza Name  Number of Clients
7                     The Classic Deluxe Pizza               1818
0                   The Barbecue Chicken Pizza               1816
20                         The Pepperoni Pizza               1758
12                          The Hawaiian Pizza               1736
30                      The Thai Chicken Pizza               1721
4                 The California Chicken Pizza               1685
23                          The Sicilian Pizza               1508
26                     The Spicy Italian Pizza               1475
1                           The Big Meat Pizza               1453
9                        The Four Cheese Pizza               1444
14                   The Italian Supreme Pizza               1425
25                 The Southwest Chicken Pizza               1393
22            The Prosciutto and Arugula Pizza               1209
17                          The Mexicana Pizza               1170
19        